In [2]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from dotenv import load_dotenv
import os

load_dotenv()

access_token = os.getenv('ACCESS_TOKEN_HUGGING_FACE')

model = TFAutoModelForSequenceClassification.from_pretrained("krisschaaf/bert_german_fake_news_1", token=access_token)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# make model return hidden states:
model.config.output_hidden_states = True

Some layers from the model checkpoint at krisschaaf/bert_german_fake_news_1 were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at krisschaaf/bert_german_fake_news_1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [3]:
import pandas as pd

df = pd.read_csv('../datasets/combined/combined_dataset_200samples_50-50split.csv')
first_article = df.iloc[0]['article']

print(f"First line of the dataset: {first_article}")

First line of the dataset: Rot-Weiss Essen besiegt Zweitligist Düsseldorf und zieht als einziger Viertligist ins Achtelfinale ein. Wolfsburg, Stuttgart und Bremen gewinnen. Der frühere Bundesligist Rot-Weiss Essen steht nach der Überraschung im Nachbarschafts-Duell mit Fortuna Düsseldorf erstmals seit zwölf Jahren wieder im DFB-Pokal-Achtelfinale. Der bislang noch ungeschlagene Spitzenreiter der Regionalliga West kam am Mittwoch zu einem glücklichen 3:2 (2:1) in der zweiten Pokalrunde gegen den Absteiger aus der Fußball-Bundesliga. Simon Engelmann (15. Minute), Kapitän Marco Kehl-Gomez (39.) und der eingewechselte Oguzhan Kefkir (70.) nutzten die einzigen drei echten Chancen des Viertligisten eiskalt aus.
Die deutlich feldüberlegene Fortuna kam trotz ihres höheren Aufwands nur zum zwischenzeitlichen 1:1 und zum späten Elfmeter-Tor jeweils durch Torjäger Rouwen Hennings (36./87.). Damit scheiterte das Team von Trainer Uwe Rösler wie schon in der Vorsaison an einem Viertligisten. Im März

In [4]:
max_length = 0 

for article in df['article']:
    input_ids = tokenizer.encode(article, add_special_tokens=True, truncation=True, max_length=512)

    max_length = max(max_length, len(input_ids))

print(f"Max length of input IDs: {max_length}")


Max length of input IDs: 512


In [5]:
import tensorflow as tf

input_ids = []
attention_masks = []

for article in df['article']:
    encoded_dict = tokenizer.encode_plus(
        article,
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf',
        padding='max_length'  # Ensure all sequences are of the same length
    )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)

print(f"Original first article: {first_article}")
print(f"Tokenized first article: {input_ids[0]}")

print(f"Length of original first article: {len(first_article)}")
print(f"Length of tokenized first article: {len(input_ids[0])}")

Original first article: Rot-Weiss Essen besiegt Zweitligist Düsseldorf und zieht als einziger Viertligist ins Achtelfinale ein. Wolfsburg, Stuttgart und Bremen gewinnen. Der frühere Bundesligist Rot-Weiss Essen steht nach der Überraschung im Nachbarschafts-Duell mit Fortuna Düsseldorf erstmals seit zwölf Jahren wieder im DFB-Pokal-Achtelfinale. Der bislang noch ungeschlagene Spitzenreiter der Regionalliga West kam am Mittwoch zu einem glücklichen 3:2 (2:1) in der zweiten Pokalrunde gegen den Absteiger aus der Fußball-Bundesliga. Simon Engelmann (15. Minute), Kapitän Marco Kehl-Gomez (39.) und der eingewechselte Oguzhan Kefkir (70.) nutzten die einzigen drei echten Chancen des Viertligisten eiskalt aus.
Die deutlich feldüberlegene Fortuna kam trotz ihres höheren Aufwands nur zum zwischenzeitlichen 1:1 und zum späten Elfmeter-Tor jeweils durch Torjäger Rouwen Hennings (36./87.). Damit scheiterte das Team von Trainer Uwe Rösler wie schon in der Vorsaison an einem Viertligisten. Im März ha

In [6]:
output = model(input_ids, attention_mask=attention_masks)

In [7]:
output.hidden_states 

(<tf.Tensor: shape=(200, 512, 768), dtype=float32, numpy=
 array([[[-0.48230153,  0.32629097,  0.37618014, ...,  0.28631747,
          -0.90260947,  0.01283996],
         [-1.9574678 ,  0.11998283, -0.52765363, ..., -1.0067387 ,
          -0.75383043, -0.2297786 ],
         [-1.3107111 ,  2.3562386 ,  1.7727066 , ...,  0.44063115,
           0.30648267, -0.75741875],
         ...,
         [ 1.9676633 , -0.23459473,  1.3176453 , ..., -0.19063394,
          -0.9960441 , -0.3939369 ],
         [-0.32624888,  0.31903368,  1.6460272 , ...,  0.36437565,
           0.18716516,  0.39599824],
         [-0.51195425,  0.46750206,  0.5430804 , ...,  0.03593849,
          -1.0480646 , -1.6619052 ]],
 
        [[-0.48230153,  0.32629097,  0.37618014, ...,  0.28631747,
          -0.90260947,  0.01283996],
         [ 0.5708568 ,  1.113294  , -0.01494729, ...,  0.7593649 ,
          -0.3991577 , -0.00854908],
         [ 1.1703435 ,  1.3008618 , -0.4110194 , ..., -0.8564349 ,
           0.87139624,  0.